In [ ]:
!pip install numpy
!pip install tensorflow
!pip install opencv-python-headless
!pip install torch
!pip install Pillow
!pip install torchsummaryX

In [11]:
import os
import numpy as np
from numpy import linalg as LA
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Softmax,Input,MaxPooling2D,Dropout,Dense,Conv2D,Flatten,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import metrics

In [2]:
len_y = len(os.listdir ('images'))
def load_img(f):
    f=open(f)
    lines=f.readlines()
    imgs, lab = [], []
    for i in range(len(lines)):
        fn, label = lines[i].split(' ')
        
        im1=cv2.imread(fn)
        im1=cv2.resize(im1, (128,128))
        imgs.append(im1)
        lab.append(int(label))
        
    imgs= np.asarray(imgs, np.float32)
    lab= np.asarray(lab, np.int32)
    return imgs, np.eye(len_y)[lab],lab

x,y,sim_y = load_img('train.txt')
vx,vy,sim_vy = load_img('val.txt')
tx,ty,sim_vy = load_img('test.txt')

In [3]:
class RRDB(tf.keras.Model):
    def __init__(self, K):
        super(RRDB, self).__init__()
        self.fc1 = Conv2D(K, (3, 3), activation='relu', padding='same')
        self.fc2 = Conv2D(K, (3, 3), activation='relu', padding='same')
        self.fc3 = Conv2D(K, (3, 3), activation='relu', padding='same')
        self.fcx = Conv2D(K, (1, 1), activation='relu', padding='same')
        
    def call(self, x):
        x1 = self.fc1(x) + x
        x2 = self.fc2(x1) + x1 + x
        x3 = self.fc3(x2) + x2 + x1 + x
        x4 = tf.concat([x, x1, x2, x3], -1)
        x5 = self.fcx(x4)
        x6 = x + x5
        x6 = MaxPooling2D(pool_size=(4, 4))(x6)
        return x6

class RRDB_first(RRDB):
    def __init__(self, K):
        super(RRDB_first, self).__init__(K)
        self.fc = Conv2D(K, (3, 3), activation='relu', padding='same')
        
    def call(self, inputs):
        x = self.fc(inputs)
        x1 = self.fc1(x) + x
        x2 = self.fc2(x1) + x1 + x
        x3 = self.fc3(x2) + x2 + x1 + x
        x4 = tf.concat([x, x1, x2, x3], -1)
        x5 = self.fcx(x4)
        x6 = x + x5
        x6 = MaxPooling2D(pool_size=(4, 4))(x6)
        return x6

def net(first_kernel = 64,first_flat = 1024):
    input_x = Input(shape=(128,128,3))
    x = RRDB_first(first_kernel)(input_x)
    x = RRDB(first_kernel)(x)
    x = RRDB(first_kernel)(x)
    x = Flatten()(x)
    x = Dense(first_flat, activation='relu')(x)
    x = Dense(len_y)(x)
    model = Model(inputs=input_x, outputs=x)
    return model

In [4]:
def get_flops(model, model_inputs) -> float:
        if not isinstance(
            model, (tf.keras.models.Sequential, tf.keras.models.Model)
        ):
            raise ValueError(
                "Calculating FLOPS is only supported for "
                "`tf.keras.Model` and `tf.keras.Sequential` instances."
            )

        from tensorflow.python.framework.convert_to_constants import (
            convert_variables_to_constants_v2_as_graph,
        )

        # Compute FLOPs for one sample
        batch_size = 1
        inputs = [
            tf.TensorSpec([batch_size] + list(inp.shape[1:]), inp.dtype)
            for inp in model_inputs
        ]

        # convert tf.keras model into frozen graph to count FLOPs about operations used at inference
        real_model = tf.function(model).get_concrete_function(inputs)
        frozen_func, _ = convert_variables_to_constants_v2_as_graph(real_model)

        # Calculate FLOPs with tf.profiler
        run_meta = tf.compat.v1.RunMetadata()
        opts = (
            tf.compat.v1.profiler.ProfileOptionBuilder(
                tf.compat.v1.profiler.ProfileOptionBuilder().float_operation()
            )
            .with_empty_output()
            .build()
        )

        flops = tf.compat.v1.profiler.profile(
            graph=frozen_func.graph, run_meta=run_meta, cmd="scope", options=opts
        )

        tf.compat.v1.reset_default_graph()

        # convert to GFLOPs
        return (flops.total_float_ops / 1e9)/2

In [5]:
model = net(first_kernel = 512,first_flat = 2048)
my_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
my_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
my_metrics = [metrics.Accuracy(),metrics.Precision(name='precision'),metrics.Recall(name='recall')]
epochs = 30
batch_size = 5

model.compile(optimizer = my_optimizer, loss = my_loss, metrics = my_metrics)
dynamic_result = model.fit(x, y, epochs=epochs, batch_size=batch_size, validation_data=(vx,vy))

In [7]:
get_flops(model, [tx[0:1]])

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


142.242927641

In [5]:
def evaluate(model,tx,ty,print_stuff = ""):
    sum_predict = 0
    for i in range(len(tx)):
        input_x = tx[i:i+1]
        y_ = model(input_x)
        if(np.argmax(y_[0]) == np.argmax(ty[i])):
            sum_predict += 1
    print(print_stuff,round(sum_predict/len(tx),4),sum_predict,len(tx))
evaluate(model,tx,ty,print_stuff = "")

 0.4556 205 450


In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import random
pre_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=False)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pre_model.parameters(),lr=0.0001)
epochs = 30
class random_seed_class():
    random_seed_int = 0
    def __init__(self):
        random_seed_int = random.seed(620)
        self.update_seed()
    def update_seed(self):
        self.random_seed_int = random.randint(1, 10000)
    def shuffle(self,*args,update = False):
        for arg in args:
            np.random.seed(self.random_seed_int)
            np.random.shuffle(arg)
        if(update):
            self.update_seed()
def custom_fit(model, X, Y, num_epochs):
    Dealer = random_seed_class()
    for epoch in range(num_epochs):
        Dealer.shuffle(X,Y,update = True)
        for i in range(int(len(X)/5)):
            inputs = torch.from_numpy(np.swapaxes(x[i:i+5], 1, 3))
            targets = torch.from_numpy((Y[i:i+5])).to(torch.long)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            if(i%1000==0):
                print(f"Epoch [{epoch+1} {i*5 + 5}sample")
        print(f"Epoch {epoch+1}/{num_epochs}], Loss: {loss.item()}")
custom_fit(pre_model,x,sim_y,epochs)

Using cache found in /home/fintechuser/.cache/torch/hub/pytorch_vision_v0.10.0


In [16]:
def evaluate(model,tx,ty,print_stuff = ""):
    sum_predict = 0
    for i in range(len(tx)):
        input_x = torch.from_numpy(np.swapaxes(tx[i:i+1], 1, 3))
        y_ = model(input_x)
        if(np.argmax(y_[0].detach().numpy()) == np.argmax(ty[i])):
            sum_predict += 1
    print(print_stuff,round(sum_predict/len(tx),4),sum_predict,len(tx))
evaluate(pre_model,tx,ty,print_stuff = "")

 0.0 0 450


In [17]:
pytorch_total_params = sum(p.numel() for p in pre_model.parameters() if p.requires_grad)
pytorch_total_params

21797672

In [27]:
from torchsummaryX import summary
summary(pre_model,torch.from_numpy(np.swapaxes(tx[0:1], 1, 3)))